In [4]:
import sys
sys.path.append('/notebook/shared/extra/')
import statsmodels

In [2]:
import scipy.stats.stats as stats

In [6]:
import pandas as pd
import math

In [7]:
from impala.dbapi import connect
from impala.util import as_pandas
import re 
#get data
def get_data(sql):
    conn = connect(host = '10.1.169.3',port = 10000, auth_mechanism = 'PLAIN')
    cursor = conn.cursor()
    cursor.execute(sql)
    df = as_pandas(cursor)
    return df


In [8]:
df= get_data('select * from tmp.cebai_all_data_project1_datasource_3')

In [208]:
df_copy = df.copy()
df_copy.drop(['labels','aid','uid','dt','direct_contact_score','mob_score','contacts_relatives_redline_cnt2','contacts_relatives_redline_cnt1','n_91overduetimes_cnt','rong360_fuzzy_black_hit'],axis = 1,inplace = True)
df_copy.to_csv('df_copy_0204.csv',index = False)

In [186]:
print(len(df))

493743


In [10]:

import pandas as pd
def dropna_all(df):
    df_copy = df.copy()
    df_copy = df_copy.dropna(subset = ['direct_contact_score','phone_gray_score','mob_score','honeypot_model_score','contacts_relatives_redline_cnt2','contacts_relatives_redline_cnt','contacts_loankeywords_redline_cnt','contacts_relatives_redline_cnt1','tongdun_score','n_91overduetimes_cnt','n_91applytimes_cnt','n_91rejectprecent','rong360_fuzzy_black_hit','ant_score','ar_score'],
                    how ='all')
    return df_copy
def dropna_any(df,how_str):
    df_copy = df.copy()
    df_copy = df_copy.dropna(subset = ['phone_gray_score','honeypot_model_score','contacts_relatives_redline_cnt','contacts_loankeywords_redline_cnt','tongdun_score','n_91overduetimes_cnt','n_91applytimes_cnt','n_91rejectprecent','rong360_fuzzy_black_hit','ant_score','ar_score'],
                    how =how_str)
    return df_copy

In [194]:
def dropna_test(df):
    df_copy = df.copy()
    df_copy = df_copy.dropna(subset = ['phone_gray_score','honeypot_model_score','contacts_relatives_redline_cnt','contacts_loankeywords_redline_cnt','tongdun_score','n_91applytimes_cnt','n_91rejectprecent','ant_score','ar_score'],
                    how ='all')
    return df_copy

In [11]:
df_drop = dropna_all(df)

In [200]:
print(len(df_drop))
print(df_drop.size)
print(df_drop.columns)

214656
4078464
Index(['labels', 'aid', 'uid', 'dt', 'direct_contact_score',
       'phone_gray_score', 'mob_score', 'honeypot_model_score',
       'contacts_relatives_redline_cnt2', 'contacts_relatives_redline_cnt',
       'contacts_loankeywords_redline_cnt', 'contacts_relatives_redline_cnt1',
       'tongdun_score', 'n_91overduetimes_cnt', 'n_91applytimes_cnt',
       'n_91rejectprecent', 'rong360_fuzzy_black_hit', 'ant_score',
       'ar_score'],
      dtype='object')


In [195]:
df_drop_test = dropna_test(df)

In [187]:
print(len(df_drop[(df_drop.dt >= '2017-02-01') & (df_drop.dt <'2017-04-01')]))

83825


In [197]:
print(len(df_drop_test[(df_drop_test.dt >= '2017-02-01') & (df_drop_test.dt <'2017-04-01')]))

83825


In [13]:
df_drop_any = dropna_any(df_drop,'any')
print(df_drop_any.size)

1527923


In [205]:
#good_bad
def get_good_bad_num(df):
    #?! size and len
    good_num = len(df[df['labels'] == 0])
    bad_num = len(df[df['labels'] == 1])
    return float(good_num),float(bad_num)
#get_everyday_num
def get_oneday_num(df,time):
    num = len(df[df['dt'] == time])
    return num
def get_everyday_num(df):
    nums =[]
    times = pd.date_range('20170201','20170415')
    for time in times:
        time = str(time).split(' ')[0]
        num = get_oneday_num(df,time)
        temp = [time,num]
        nums.append(temp)
    return nums
    

In [202]:
get_everyday_num(df_drop)

[['2017-02-01', 7],
 ['2017-02-02', 8],
 ['2017-02-03', 13],
 ['2017-02-04', 9],
 ['2017-02-05', 5],
 ['2017-02-06', 13],
 ['2017-02-07', 14],
 ['2017-02-08', 10],
 ['2017-02-09', 14],
 ['2017-02-10', 15],
 ['2017-02-11', 12],
 ['2017-02-12', 29],
 ['2017-02-13', 19],
 ['2017-02-14', 16],
 ['2017-02-15', 11],
 ['2017-02-16', 20],
 ['2017-02-17', 22],
 ['2017-02-18', 17],
 ['2017-02-19', 16],
 ['2017-02-20', 24],
 ['2017-02-21', 27],
 ['2017-02-22', 17],
 ['2017-02-23', 24],
 ['2017-02-24', 18],
 ['2017-02-25', 26],
 ['2017-02-26', 22],
 ['2017-02-27', 21],
 ['2017-02-28', 23],
 ['2017-03-01', 28],
 ['2017-03-02', 24],
 ['2017-03-03', 32],
 ['2017-03-04', 33],
 ['2017-03-05', 31],
 ['2017-03-06', 54],
 ['2017-03-07', 74],
 ['2017-03-08', 99],
 ['2017-03-09', 134],
 ['2017-03-10', 177],
 ['2017-03-11', 176],
 ['2017-03-12', 229],
 ['2017-03-13', 223],
 ['2017-03-14', 261],
 ['2017-03-15', 276],
 ['2017-03-16', 359],
 ['2017-03-17', 518],
 ['2017-03-18', 686],
 ['2017-03-19', 1395],
 ['20

In [15]:
#pickle
import pickle
def to_pickle():
    pickle.dump(df_drop, open("df_drop.pkl", 'wb'))

In [16]:
#to_csv
def to_csv(df,start,end):
    df_online = get_df_time(df,start,end)
    df_online.drop(df_online.columns[[1,2,3]],axis = 1,inplace = True)
    df_online.to_csv('datatrain_new.csv')
    return df_online

In [17]:
# caculate the mounts of nan
def missing_values_table(df): 
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum()/len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        return mis_val_table_ren_columns

In [18]:
import pandas as pd
def get_df_time(df,start,end):
    df_copy = df.copy()
    df_copy = df_copy[(df_copy.dt >=start) & (df_copy.dt <end)]
    return df_copy

In [179]:
def col_names():
    col_names = []
    #add time
    #col_names = ['dt']
    for i in range(1,16):
        col_name = "F"+str(i)
        col_names.append(col_name)
    return col_names
def get_Y_X(df):
    df_copy = df.copy()
    df_Y = df_copy['labels'].to_frame(name = 'Y')
    #del df_copy[['labels','aid','uid','dt']]
    #save dt
    #df_Y['Y'] = df_Y['Y'].map(convert_labels)
    df_copy.drop(df.columns[[0,1,2,3]],axis=1, inplace=True)
    df_X = df_copy
    df_X.columns = col_names()
    return df_Y,df_X
#map categorical variable
def map_X(df_x):
    return df_x
def drop_X(df_x):
    del df_x['F1']
    del df_x['F3']
    del df_x['F5']
    del df_x['F8']
    del df_x['F10']
    del df_x['F13']
    return df_x
def get_fix_X(df_x):
    #df_x = map_X(df_x)
    df_x = drop_X(df_x)
    #df_x = dummies_X_all(df_x)
    df_x = fillna(df_x)
    return df_x

In [20]:
#nomolize
from sklearn import preprocessing
def normalize(df):
    min_max_scaler = preprocessing.MinMaxScaler()
    np_scaled = min_max_scaler.fit_transform(df)
    df_normalized = pd.DataFrame(np_scaled)
    return df_normalized

In [21]:
def fillna(df):
    return df.fillna(9999)

In [22]:
#RF_train
from scipy.stats import ks_2samp
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.metrics import roc_auc_score,roc_curve
import matplotlib.pyplot as plot
import numpy as np
from math import sqrt,fabs,exp
def train_online(df):
    df_online = get_df_time(df,'2017-02-01','2017-04-01')
    y,X = get_Y_X(df_online)
    X = get_fix_X(X)
    #y = normalize(y)
    #X = normalize(X)
    xTrain,xTest,yTrain,yTest = train_test_split(X.values,y.values.ravel(),test_size = 0.2,random_state = 520)
    rf = ensemble.RandomForestClassifier(n_estimators=4000,max_depth=7,oob_score = True,n_jobs = -1,random_state = 531,class_weight = 'auto')
    rf.fit(xTrain,yTrain)
    print("online----------oob_score:%f" % rf.oob_score_)
    prediction = rf.predict_proba(xTest)
    auc = roc_auc_score(yTest,prediction[:,1:2])
    print ("online-------AUC Score (test):%f" % auc)
    return rf

In [181]:
rf = train_online(df_drop)

online----------oob_score:0.610856
online-------AUC Score (test):0.601861


In [23]:
#LR_train
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,roc_curve
def train_lr(df):
    df_online = get_df_time(df,'2017-02-01','2017-04-01')
    y,X = get_Y_X(df_online)
    X = get_fix_X(X)
    #y = normalize(y)
    X = normalize(X)
    xTrain,xTest,yTrain,yTest = train_test_split(X.values,y.values.ravel(),test_size = 0.2,random_state = 520)
    lr = LogisticRegression(C = 500.0,random_state = 0,class_weight = 'balanced')
    lr.fit(xTrain,yTrain)
    prediction = lr.predict_proba(xTest)
    #print(prediction[:,1:2])
    auc = roc_auc_score(yTest,prediction[:,1:2])
    print ("online-------AUC Score (test):%f" % auc)
    return lr
    
    

In [128]:
def get_df_woe(df):
    df_copy = df.copy()
    #add phone_gray_score 5 and contacts_relatives_redline 9
    #del ar_score
    df_copy.drop(df_copy.columns[[1,2,3,4,5,6,8,9,11,13,16]],axis=1, inplace=True)
    return df_copy
def get_train_test_from_df(df):
    msk = np.random.rand(len(df)) < 0.8
    train = df[msk]
    test = df[~msk]
    return train,test
def change_df_numeric(df):
    return df.apply(pd.to_numeric,errors='ignore')
def df_fillna(df):
    #add code
    col = ["honeypot_model_score","contacts_loankeywords_redline_cnt","tongdun_score","ar_score"]
    df[col] = df[col].fillna(-9999)
    df['n_91applytimes_cnt'] = df['n_91applytimes_cnt'].fillna(2)
    df['n_91rejectprecent'] = df['n_91rejectprecent'].fillna(0)
    df['ant_score'] = df['ant_score'].fillna(300)
    return df
def get_df_woe_train_test(df):
    df_online = get_df_time(df,'2017-02-01','2017-04-01')
    df_online = get_df_woe(df_online)
    df_online = change_df_numeric(df_online)
    #fillna
    #df_online.fillna(-9999,inplace=True)
    df_online = df_fillna(df_online)
    train,test = get_train_test_from_df(df_online)
    return train,test

In [129]:
#print(df_drop.head(5))
test = get_df_woe(df_drop)
test = df_fillna(test)
print(test.head(5))

      labels honeypot_model_score contacts_loankeywords_redline_cnt  \
174        0                -9999                                 0   
511        0                -9999                                 1   
941        0                -9999                             -9999   
1058       0                -9999                                 0   
1143       1                  628                                 1   

            tongdun_score n_91applytimes_cnt n_91rejectprecent ant_score  \
174   0.09842409198988074                  0               0.0       566   
511   0.09842409198988074                  0               0.0       597   
941                0.1057                  0               0.0       622   
1058              0.10094                  0               0.0       634   
1143              0.10044                  1               1.0        -1   

     ar_score  
174     -9999  
511     -9999  
941     -9999  
1058    -9999  
1143      484  


In [26]:
def get_woe(df,colname,p,q):
    df = change_df_numeric(df)
    total_good,total_bad = get_good_bad_num(df)
    df_p_q = df[(df[colname]>p) & (df[colname]<=q)]
    good,bad = get_good_bad_num(df_p_q)
    obs = good + bad
    bad_rate = bad/obs
    pop = (good+bad)/(total_good+total_bad)
    woe = math.log((bad/total_bad)/(good/total_good))
    iv = (bad/total_bad-good/total_good)*woe
    return obs,bad,woe,bad_rate,pop,iv
def get_woe_list(df,colname,cuts):
    woe_list =[]
    for i in range(len(cuts)-1):
        obs,bad,woe,bad_rate,pop,iv = get_woe(df,colname,cuts[i],cuts[i+1])
        temp = [cuts[i],cuts[i+1],obs,bad,woe,bad_rate,pop,iv]
        woe_list.append(temp)
    df_woe_list = pd.DataFrame(woe_list,columns=['start','end','obs','bad','woe','badrate','pop','iv'])
    return df_woe_list

In [203]:
#del
F2 = 'phone_gray_score'
cuts = [-float('Inf'),0,50,75,float('Inf')]
woe_F2 = get_woe_list(train,F2,cuts)
woe_F2.to_csv("woe_F2.csv")
print(woe_F2)

KeyError: 'phone_gray_score'

In [204]:
F4 = 'honeypot_model_score'
cuts = [-float('Inf'),-9999,610,630,640,float('Inf')]
woe_F4 = get_woe_list(train,F4,cuts)
#print(cuts)
woe_F4.to_csv("woe_F4.csv")
print(woe_F4)

         start          end      obs     bad       woe   badrate       pop  \
0         -inf -9999.000000  28035.0  2556.0 -0.105196  0.091172  0.417697   
1 -9999.000000   610.000000   9873.0  1350.0  0.351551  0.136737  0.147099   
2   610.000000   630.000000  12798.0  1544.0  0.207868  0.120644  0.190679   
3   630.000000   640.000000   7348.0   653.0 -0.133324  0.088868  0.109479   
4   640.000000          inf   9064.0   627.0 -0.405220  0.069175  0.135046   

         iv  
0  0.004432  
1  0.020902  
2  0.008951  
3  0.001845  
4  0.018862  


In [31]:
#del
F6 = "contacts_relatives_redline_cnt"
cuts = [-float('Inf'),-9999,10,15,30,float('Inf')]
woe_F6 = get_woe_list(train,F6,cuts)
woe_F6.to_csv('woe_F6.csv')
print(woe_F6)

         start          end       obs      bad       woe   badrate       pop  \
0         -inf -9999.000000  190420.0  16160.0 -0.189393  0.084865  0.283583   
1 -9999.000000    10.000000  114610.0  12970.0  0.129818  0.113166  0.170683   
2    10.000000    15.000000   81230.0   9140.0  0.123362  0.112520  0.120972   
3    15.000000    30.000000  154930.0  16440.0  0.057536  0.106112  0.230729   
4    30.000000          inf  130290.0  12960.0 -0.014506  0.099470  0.194034   

         iv  
0  0.009431  
1  0.003029  
2  0.001934  
3  0.000782  
4  0.000041  


In [206]:
F7 = "contacts_loankeywords_redline_cnt"
cuts_F7 = [-float('Inf'),-9999,1,float('Inf')]
woe_F7 = get_woe_list(train,F7,cuts_F7)
woe_F7.to_csv("woe_F7.csv")
print(woe_F7)

         start          end      obs     bad       woe   badrate       pop  \
0         -inf -9999.000000  19101.0  1580.0 -0.211760  0.082718  0.284588   
1 -9999.000000     1.000000  37227.0  4003.0  0.077987  0.107529  0.554650   
2     1.000000          inf  10790.0  1147.0  0.065133  0.106302  0.160762   

         iv  
0  0.011725  
1  0.003480  
2  0.000700  


In [207]:
F9 = "tongdun_score"
cuts_F9 = [-float('Inf'),0.11,0.13,0.15,0.20,float('Inf')]
woe_F9 = get_woe_list(train,F9,cuts_F9)
woe_F9.to_csv("woe_F9.csv")
print(woe_F9)

      start       end      obs     bad       woe   badrate       pop        iv
0      -inf  0.110000  20229.0  1697.0 -0.196422  0.083889  0.301395  0.010750
1  0.110000  0.130000  12137.0  1025.0 -0.189118  0.084453  0.180831  0.005996
2  0.130000  0.150000  14804.0  1564.0  0.058219  0.105647  0.220567  0.000765
3  0.150000  0.200000  11743.0  1353.0  0.155696  0.115218  0.174961  0.004513
4  0.200000       inf   8205.0  1091.0  0.319245  0.132968  0.122247  0.014144


In [170]:
F11 = "n_91applytimes_cnt"
cuts_F11 = [-float('Inf'),1,float('Inf')]
woe_F11 = get_woe_list(train,F11,cuts_F11)
woe_F11.to_csv("woe_F11.csv")
print(woe_F11)

      start       end       obs      bad       woe   badrate       pop  \
0      -inf  1.000000  366896.0  34368.0 -0.075383  0.093672  0.683304   
1  1.000000       inf  170048.0  19472.0  0.148725  0.114509  0.316696   

         iv  
0  0.003768  
1  0.007433  


In [171]:
F12 = "n_91rejectprecent"
cuts_F12 = [-float('Inf'),0.1,0.8,float('Inf')]
woe_F12 = get_woe_list(train,F12,cuts_F12)
woe_F12.to_csv("woe_F12.csv")
print(woe_F12)

      start       end       obs      bad       woe   badrate       pop  \
0      -inf  0.100000  303888.0  27120.0 -0.128693  0.089243  0.565958   
1  0.100000  0.800000   85928.0   9776.0  0.141414  0.113770  0.160032   
2  0.800000       inf  147128.0  16944.0  0.155180  0.115165  0.274010   

         iv  
0  0.008903  
1  0.003386  
2  0.007020  


In [172]:
F14 = 'ant_score'
cuts_F14 = [-float('Inf'),0,600,630,660,680,float('Inf')]
woe_F14 = get_woe_list(train,F14,cuts_F14)
woe_F14.to_csv("woe_F14.csv")
print(woe_F14)

        start         end       obs      bad       woe   badrate       pop  \
0        -inf    0.000000  237536.0  22968.0 -0.040309  0.096693  0.442385   
1    0.000000  600.000000   66376.0   9424.0  0.395266  0.141979  0.123618   
2  600.000000  630.000000   83152.0   9064.0  0.093272  0.109005  0.154862   
3  630.000000  660.000000   76720.0   7200.0 -0.073318  0.093848  0.142883   
4  660.000000  680.000000   34856.0   2880.0 -0.212980  0.082626  0.064916   
5  680.000000         inf   38304.0   2304.0 -0.554657  0.060150  0.071337   

         iv  
0  0.000707  
1  0.022589  
2  0.001398  
3  0.000746  
4  0.002704  
5  0.017596  


In [173]:
F15 = 'ar_score'
cuts_F15 = [-float('Inf'),-9999,500,560,float('Inf')]
woe_F15 = get_woe_list(train,F15,cuts_F15)
woe_F15.to_csv("woe_F15.csv")
print(woe_F15)

         start          end       obs      bad       woe   badrate       pop  \
0         -inf -9999.000000  247640.0  22192.0 -0.124142  0.089614  0.461203   
1 -9999.000000   500.000000   95048.0  11552.0  0.216275  0.121539  0.177017   
2   500.000000   560.000000  125704.0  13312.0  0.060888  0.105900  0.234110   
3   560.000000          inf   68552.0   6784.0 -0.014603  0.098961  0.127671   

         iv  
0  0.006763  
1  0.009025  
2  0.000889  
3  0.000027  


In [152]:
#change the dataframe to woe
def convert_woe_F2(data):
    if data<=0:
        out = -0.067800
    elif data >0 and data <=50:
        out = 0.149807
    elif data >50 and data <=75:
        out = 0.059014
    elif data >75:
        out = -0.036569
    return out
def convert_woe_F4(data):
    #分段法进行表示
    if data<=-9999:
        out = -0.105196
    elif data >-9999 and data<=610:
        out = 0.351551
    elif (data>610) and (data <=630):
        out = 0.207868
    elif data>630 and data <= 640:
        out = -0.133324
    elif data>640:
        out = -0.405220
    return out
def convert_woe_F6(data):
    if data<=-9999:
        out = -0.189393
    elif data >-9999 and data <=10:
        out = 0.129818
    elif data >10 and data <=15:
        out = 0.123362
    elif data >15 and data <=30:
        out = 0.057536
    elif data >30:
        out = -0.014506
    return out
def convert_woe_F7(data):
    #分段法进行表示
    if data <=-9999:
        out = -0.211760
    elif data >-9999 and data<=1:
        out = 0.077987
    elif data>1:
        out = 0.065133
    return out
def convert_woe_F9(data):
    #分段法进行表示
    if data <= 0.11:
        out = -0.196422
    elif data >0.11 and data<=0.13:
        out = -0.189118
    elif data>0.13 and data <=0.15:
        out = 0.058219
    elif data>0.15 and data <=0.20:
        out = 0.155696
    elif data>0.20:
        out = 0.319245
    return out
def convert_woe_F11(data):
    #分段法进行表示
    if data <= 1:
        out = -0.075383
    elif data>1:
        out = 0.148725
    return out
def convert_woe_F12(data):
    #分段法进行表示
    if data <=0.1:
        out = -0.128693
    elif data>0.1 and data <= 0.8:
        out = 0.141414
    elif data>0.8:
        out = 0.155180
    return out
def convert_woe_F14(data):
    #分段法进行表示
    if data <=0:
        out = -0.040309
    elif data>0 and data<= 600:
        out = 0.395266
    elif data>600 and data<=630:
        out = 0.093272
    elif data>630 and data<=660:
        out = -0.073318
    elif data >660 and data <=680:
        out = -0.212980
    elif data>680:
        out = -0.554657
    return out
def convert_woe_F15(data):
    #分段法进行表示
    if data <=-9999:
        out = -0.124142
    if data>-9999 and data<=500:
        out = 0.216275
    elif data>500 and data<=560:
        out = 0.060888
    elif data>560:
        out = -0.014603
    return out
def woe_insert_df(df):
    #每一列进行转换
    #df['phone_gray_score'] = df["phone_gray_score"].map(convert_woe_F2)
    df["honeypot_model_score"] = df["honeypot_model_score"].map(convert_woe_F4)
    #df["contacts_relatives_redline_cnt"] = df["contacts_relatives_redline_cnt"].map(convert_woe_F6)
    df["contacts_loankeywords_redline_cnt"] = df["contacts_loankeywords_redline_cnt"].map(convert_woe_F7)
    df["tongdun_score"] = df["tongdun_score"].map(convert_woe_F9)
    df["n_91applytimes_cnt"] = df["n_91applytimes_cnt"].map(convert_woe_F11)
    df["n_91rejectprecent"] = df["n_91rejectprecent"].map(convert_woe_F12)
    df["ant_score"] = df["ant_score"].map(convert_woe_F14)
    df["ar_score"] = df["ar_score"].map(convert_woe_F15)
    #drop n_91applytimes_cnt
    #df.drop('n_91applytimes_cnt',axis=1,inplace = True)
    return df

In [153]:
#model_train
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,roc_curve
def train_lr_woe_get():
    #train,test = get_df_woe_train_test(df)
    #golable values train and test
    train_copy = train.copy()
    test_copy = test.copy()
    train_woe = woe_insert_df(train_copy)
    test_woe = woe_insert_df(test_copy)
    yTrain = train_woe['labels'].to_frame().values.ravel()
    train_woe.drop(train_woe.columns[[0]],axis=1,inplace=True)
    xTrain = train_woe.values
    yTest = test_woe['labels'].to_frame().values.ravel()
    test_woe.drop(test_woe.columns[[0]],axis=1,inplace=True)
    xTest = test_woe.values
    return xTrain,xTest,yTrain,yTest
def train_lr_woe():
    xTrain,xTest,yTrain,yTest = train_lr_woe_get()
    lr = LogisticRegression(C = 1.0,random_state = 0,class_weight = 'balanced')
    #select_feature = lr.transform(xTrain)
    #print(select_feature)
    lr.fit(xTrain,yTrain)
    prediction = lr.predict_proba(xTest)
    #print(prediction[:,1:2])
    auc = roc_auc_score(yTest,prediction[:,1:2])
    print ("online-------AUC Score (test):%f" % auc)
    return lr

In [154]:
import statsmodels.formula.api as smf
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve
def get_data_forward_selected():
    train_copy = train.copy()
    test_copy = test.copy()
    train_woe = woe_insert_df(train_copy)
    test_woe = woe_insert_df(test_copy)
    return train_woe,test_woe
def forward_selected(data,response):
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score,best_new_score = 0.0,0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,'+'.join(selected+[candidate]))
            score = smf.ols(formula,data).fit().rsquared_adj
            scores_with_candidates.append((score,candidate))
        scores_with_candidates.sort()
        best_new_score,best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} +1".format(response,'+'.join(selected))
    model = smf.ols(formula,data).fit()
    return model
def train_forward_selected():
    train,test = get_data_forward_selected()
    model = forward_selected(train,'labels')
    yTest = test['labels'].to_frame().values.ravel()
    pred = model.predict(test)
    auc = roc_auc_score(yTest,pred)
    print ("online-------AUC Score (test):%f" % auc)
    return model

In [54]:
%matplotlib inline

In [157]:
#Ridge Regression
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
def get_y_x_ridge_regression(df):
    df_online = get_df_time(df,'2017-02-01','2017-04-01')
    df_online = get_df_woe(df_online)
    df_online = change_df_numeric(df_online)
    df_online = df_fillna(df_online)
    data = df_online.copy()
    data = woe_insert_df(data)
    y = data['labels'].to_frame().values.ravel()
    data.drop(data.columns[[0]],axis=1,inplace=True)
    X= data.values
    return X,y
def plot_alpha_ridge_regression(df):
    #get y x
    X,y = get_y_x_ridge_regression(df)
    #show
    n_alphas = 200
    alphas = np.logspace(-10,-2,n_alphas)
    clf = linear_model.Ridge(fit_intercept = False)
    coefs = []
    for a in alphas:
        clf.set_params(alpha = a)
        clf.fit(X,y)
        coefs.append(clf.coef_)
    ax = plt.gca()
    ax.plot(alphas,coefs)
    ax.set_xscale('log')
    ax.set_xlim(ax.get_xlim()[::-1])
    plt.xlabel('alpha')
    plt.ylabel('weights')
    plt.axis('tight')
    plt.show()
def best_alpha(df):
    X,y = get_y_x_ridge_regression(df)
    clf = linear_model.RidgeCV(alphas=[0.1,1.0,8.0,9.0,10.0,11.0,20.0,30.0,40.0,45.0,50.0,51.0,60.0,70.0,100.0,1000.0])
    clf.fit(X,y)
    #linear_model.RidgeCV(alphas=[0.1,1.0,8.0,9.0,10.0,11.0,20.0,30.0,40.0,45.0,50.0,51.0,60.0,70.0,100.0,1000.0],cv =None,fit_intercept = True,scoring = None,normalize = False)
    return clf.alpha_
def sigmoid(inX):  
    return 1.0 / (1 + np.exp(-inX))
def train_ridge_regression():
    xTrain,xTest,yTrain,yTest = train_lr_woe_get()
    clf = linear_model.Ridge(fit_intercept = True)
    clf.set_params(alpha =45)
    clf.fit(xTrain,yTrain)
    prediction = clf.predict(xTest)
    #prediction = sigmoid(prediction)
    print(prediction)
    auc = roc_auc_score(yTest,prediction)
    print ("online-------AUC Score (test):%f" % auc)
    return clf

In [158]:
#plot_alpha_ridge_regression(df_drop)
best_alpha(df_drop)

45.0

In [159]:
ridge = train_ridge_regression()
ridge.coef_

[ 0.11805225  0.07956226  0.04717336 ...,  0.12552514  0.14361264
  0.08671106]
online-------AUC Score (test):0.589541


array([ 0.05265962,  0.03781687,  0.0711045 ,  0.00470664,  0.0448961 ,
        0.08214107, -0.03155062])

In [175]:
model_selected = train_forward_selected()
print(model_selected.model.formula)
model_selected.params

online-------AUC Score (test):0.589731
labels ~ honeypot_model_score+ant_score+tongdun_score+n_91rejectprecent+contacts_loankeywords_redline_cnt+ar_score +1


Intercept                            0.104417
honeypot_model_score                 0.053447
ant_score                            0.083353
tongdun_score                        0.073040
n_91rejectprecent                    0.048155
contacts_loankeywords_redline_cnt    0.040774
ar_score                            -0.036369
dtype: float64

In [174]:
lr_woe_all = train_lr_woe()
lr_woe_all.coef_

online-------AUC Score (test):0.589567


array([[ 0.57755067,  0.54589567,  0.79544261,  0.0537072 ,  0.56778598,
         0.93250173, -0.42417155]])

In [353]:
lr_woe_all.coef_

array([[-0.65977649,  0.62621007,  1.07773027, -0.52507218,  0.79804137,
        -0.07358959,  0.15217059,  0.84984036, -0.13822202]])

In [333]:
xtest = [[0.087041,0.067389,-0.203955,-0.073754,0.168285,-0.063447,0.181256]]
pre = lr_woe_all_1.predict_proba(xtest)
a = lr_woe_all_1.coef_
#print(a)
sum = 0
for i in range(7):
    sum +=xtest[0][i]*a[0][i]
sum +=  lr_woe_all_1.intercept_
from math import e
print(1./(1+e**(-sum)))
#print(sum)
print(pre)

[ 0.47828455]
[[ 0.52171545  0.47828455]]


In [299]:
lr_woe_all_1 = train_lr_woe()

online-------AUC Score (test):0.591702


In [315]:
lr_woe_all_1.coef_
#lr_woe_all_1.n_iter_ 

array([[ 0.56592706,  0.5496456 ,  0.80801066, -0.06267855,  0.65678485,
         0.86047866, -0.44524436]])

In [326]:
lr_woe_all_1.intercept_

array([ 0.01173057])

In [327]:
rf = train_online(df_drop_any)

online----------oob_score:0.600694
online-------AUC Score (test):0.594595


In [36]:
lr = train_lr(df_drop_any)

online-------AUC Score (test):0.582784


In [162]:
#ks
import numpy as np
import pandas as pd
from collections import Counter
def ks_value(pred,true,n):
    score = (1.-pred)*1000
    index = score.argsort()
    score_sort = np.sort(score)
    true_sort = true[index]
    len_num = pred.size
    add = int(len_num/n)
    out = []
    for i in get_split_len(len_num,n):
        if i == get_split_len(len_num,n)[-1]:
            good_num,bad_num,total = get_good_bad(true_sort,i,-1)
            min_value,max_value = get_max_min(score_sort,i,-1)
            temp = [min_value,max_value,total,bad_num,good_num]
        else:
            good_num,bad_num,total = get_good_bad(true_sort,i,i+add)
            min_value,max_value = get_max_min(score_sort,i,i+add)
            temp = [min_value,max_value,total,bad_num,good_num]
        out.append(temp)
    return out
def list_to_dataframe(out):
    df = pd.DataFrame.from_records(out,columns = ["min_value","max_value","total","bad_num","good_num"])
    return df
def get_good_bad(true,startindex,endindex):
    if endindex == -1:
        true_index = true[startindex:]
    else:
        true_index = true[startindex:endindex]
    c = Counter([i for i in true_index])
    total = true_index.size
    return c[0],c[1],total
def get_max_min(pred,startindex,endindex):
    return pred[startindex],pred[endindex]
def get_split_len(len_num,n):
    out = [0]
    add = int(len_num/n)
    index = 0
    for i in range(n-1):
        index += add
        out.append(int(index))
    return out
def get_ksvalue_offline(rf,df,start,end,name):
    df_offline = get_df_time(df,start,end)
    y,X = get_Y_X(df_offline)
    X = get_fix_X(X)
    xTest,yTest = X.values,y.values.ravel()
    prediction = rf.predict_proba(xTest)
    pred = prediction[:,1:2].ravel()
    out = ks_value(pred,yTest,10)
    df = list_to_dataframe(out)
    df.to_csv(name)
    return df
def get_ksvalue_lr(lr,df,start,end,name):
    df_offline = get_df_time(df,start,end)
    y,X = get_Y_X(df_offline)
    X = get_fix_X(X)
    X = normalize(X)
    xTest,yTest = X.values,y.values.ravel()
    prediction = lr.predict_proba(xTest)
    #print(prediction)
    pred = prediction[:,1:2].ravel()
    #print(pred)
    out = ks_value(pred,yTest,10)
    df = list_to_dataframe(out)
    df.to_csv(name)
    return df
def get_ksvalue_lr_woe(lr,df,start,end,name):
    df_offline = get_df_time(df,start,end)
    df_offline = get_df_woe(df_offline)
    df_offline = change_df_numeric(df_offline)
    #fillna
    df_offline = df_fillna(df_offline)
    df_offline_woe = woe_insert_df(df_offline)
    yTest = df_offline_woe['labels'].to_frame().values.ravel()
    df_offline_woe.drop(df_offline_woe.columns[[0]],axis=1,inplace=True)
    xTest = df_offline_woe.values
    prediction = lr.predict_proba(xTest)
    #print(prediction)
    pred = prediction[:,1:2].ravel()
    #print(pred)
    out = ks_value(pred,yTest,10)
    df = list_to_dataframe(out)
    df.to_csv(name)
    return df
def get_ksvalue_ridge(lr,df,start,end,name):
    df_offline = get_df_time(df,start,end)
    df_offline = get_df_woe(df_offline)
    df_offline = change_df_numeric(df_offline)
    #fillna
    df_offline = df_fillna(df_offline)
    df_offline_woe = woe_insert_df(df_offline)
    yTest = df_offline_woe['labels'].to_frame().values.ravel()
    df_offline_woe.drop(df_offline_woe.columns[[0]],axis=1,inplace=True)
    xTest = df_offline_woe.values
    pred= lr.predict(xTest)
    #print(prediction)
    #print(pred)
    out = ks_value(pred,yTest,10)
    df = list_to_dataframe(out)
    df.to_csv(name)
    return df

In [163]:
#selected_forward
def get_data_selected(df,st,end):
    df_offline = get_df_time(df,st,end)
    df_offline = get_df_woe(df_offline)
    df_offline = change_df_numeric(df_offline)
    #fillna
    df_offline = df_fillna(df_offline)
    df_offline_woe = woe_insert_df(df_offline)
    return df_offline_woe
def get_ksvalue_selected_forward(model,df,start,end,name):
    df_offline = get_df_time(df,start,end)
    df_offline = get_df_woe(df_offline)
    df_offline = change_df_numeric(df_offline)
    #fillna
    df_offline = df_fillna(df_offline)
    data = woe_insert_df(df_offline)
    yTest = data['labels'].to_frame().values.ravel()
    pred = model.predict(data)
    out = ks_value(pred,yTest,10)
    df = list_to_dataframe(out)
    df.to_csv(name)
    return df

In [164]:
get_ksvalue_ridge(ridge,df_drop,'2017-02-01','2017-04-01','df1_ridge.csv')
get_ksvalue_ridge(ridge,df_drop,'2017-04-01','2017-04-05','df2_ridge.csv')
get_ksvalue_ridge(ridge,df_drop,'2017-04-05','2017-04-10','df3_ridge.csv')

,min_value,max_value,total,bad_num,good_num
0,810.885818,872.508881,4793,481,4312
1,872.508881,886.330485,4793,466,4327
2,886.330485,893.278737,4793,405,4388
3,893.278737,903.359792,4793,362,4431
4,903.359792,907.078323,4793,372,4421
5,907.078323,917.203129,4793,298,4495
6,917.203129,918.875393,4793,361,4432
7,918.875393,929.067460,4793,212,4581
8,929.067460,934.011589,4793,208,4585
9,934.011589,970.932759,4802,207,4595


In [165]:
get_ksvalue_selected_forward(model_selected,df_drop,'2017-02-01','2017-04-01','df1_selected.csv')
get_ksvalue_selected_forward(model_selected,df_drop,'2017-04-01','2017-04-05','df2_selected.csv')
get_ksvalue_selected_forward(model_selected,df_drop,'2017-04-05','2017-04-10','df3_selected.csv')

,min_value,max_value,total,bad_num,good_num
0,809.346167,872.182761,4793,485,4308
1,872.182761,886.067437,4793,476,4317
2,886.067437,893.378630,4793,390,4403
3,893.378630,903.211499,4793,364,4429
4,903.211499,907.412059,4793,372,4421
5,907.412059,916.881492,4793,301,4492
6,916.881492,918.973442,4793,285,4508
7,918.973442,929.951764,4793,280,4513
8,929.951764,934.436986,4793,212,4581
9,934.436986,972.101514,4802,207,4595


In [166]:
get_ksvalue_lr_woe(lr_woe_all,df_drop,'2017-02-01','2017-04-01','df1_woe.csv')
get_ksvalue_lr_woe(lr_woe_all,df_drop,'2017-04-01','2017-04-05','df2_woe.csv')
get_ksvalue_lr_woe(lr_woe_all,df_drop,'2017-04-05','2017-04-10','df3_woe.csv')

,min_value,max_value,total,bad_num,good_num
0,274.629073,432.943986,4793,493,4300
1,432.943986,471.265057,4793,460,4333
2,471.265057,492.449014,4793,384,4409
3,492.449014,515.601776,4793,374,4419
4,515.601776,532.793442,4793,370,4423
5,532.793442,557.206597,4793,311,4482
6,557.206597,564.438664,4793,266,4527
7,564.438664,596.145753,4793,309,4484
8,596.145753,610.538222,4793,197,4596
9,610.538222,705.461968,4802,208,4594


In [183]:
get_ksvalue_offline(rf,df_drop,'2017-02-01','2017-04-01','df1.csv')
get_ksvalue_offline(rf,df_drop,'2017-04-01','2017-04-05','df2.csv')
get_ksvalue_offline(rf,df_drop,'2017-04-05','2017-04-10','df3.csv')

,min_value,max_value,total,bad_num,good_num
0,297.519206,395.011383,4793,617,4176
1,395.011383,414.579700,4793,481,4312
2,414.579700,434.760468,4793,413,4380
3,434.760468,453.363677,4793,355,4438
4,453.363677,476.607780,4793,339,4454
5,476.607780,510.291524,4793,298,4495
6,510.291524,538.412042,4793,277,4516
7,538.412042,562.949632,4793,196,4597
8,562.949632,591.651882,4793,200,4593
9,591.651882,779.894918,4802,196,4606
